| Version | Published Date| Details |
| -- | -- | -- |
| ver.1.0.0 | 2023/8/29 | 初版 |

# Webスクレイピングをやってみよう

WindowsやMacでなにか作業をするとき，インターネットを使わずに行うことは稀でしょう。今お使いのGoogle Colaboratoryですら，インターネットを使って実現されています。

**Webスクレイピング** は，プログラムを使ってWebからコンテンツをダウンロードして処理することです。たとえばば，Googleは多数のWebスクレイピングプログラムを実行して **クローリング** を行い，検索エンジン用にWebページの索引を作っています。このStationでは，Pythonで簡単にWebスクレイピングをするのに役立つ次のようなモジュールについて学びます。

- requests
- bs4
- selenium


## `requests` モジュールを用いてWebサイトからファイルをダウンロードする

Pythonの `requests` モジュールを使うと，ネットワークに関する複雑な問題に対処することなく，Webから簡単にファイルをダウンロードできます。まずはいつものように `requests` を `import` しておきます。

In [1]:
import requests

`requests.get()` はダウンロードするURLを文字列として受け取ります。 `requests.get()` の戻り値を `type()` で調べてみると `Response` オブジェクトであることがわかります。このオブジェクトにはWebサーバーのリクエストに対するレスポンス (応答) が格納されています。

In [2]:
url = 'https://www.gutenberg.org/files/1787/1787.txt'
r = requests.get(url)

`url` に格納しているURL https://www.gutenberg.org/files/1787/1787.txt はインターネット上に公開されている「ハムレット」のテキストファイルです。このWebページに対するリクエストが成功したかどうかは `Response` オブジェクトの `statud_code` を調べればわかります。もしこの値が `requests.code.ok` であれば成功です。

In [ ]:
type(r)

In [ ]:
r.status_code == requests.codes.ok

ちなみにHTTPプロトコルで「OK」の状態コードは `200` です。「Not Found」を表す状態コードの `404` は見たこともあるでしょう。HTTPの状態コードの完全なリストとその意味は [Wikipedia](https://ja.wikipedia.org/wiki/HTTP%E3%82%B9%E3%83%86%E3%83%BC%E3%82%BF%E3%82%B9%E3%82%B3%E3%83%BC%E3%83%88%E3%82%99) を参照してください。

リクエストが成功すると，Webページがダウンロードされ `Response` オブジェクトの `text` 属性に文字列として格納されます。 `len(r.text)` を呼び出してみると211,000文字以上あることがわかります。

In [ ]:
len(r.text)

`r.text[:300]` を呼び出して，冒頭300文字を表示しましょう。もしリクエストが失敗し 「 `Failed to establish a new connection` (新たな接続の確立に失敗しました)」や「 `Max retries exceeded` (最大試行回数を超えました)」といったエラーメッセージが表示されたら，インターネットの接続を確認してください。


In [ ]:
r.text[:300]

サーバへの接続は複雑です。そのため起こりうる問題すべてを列挙することはできません。エラーメッセージを引用して検索すれば，よくある原因を見つけられるでしょう。

## エラーをチェックする

先に書いたように `Response` オブジェクトには `status_code` 属性があり `requests.codes.ok` と比較することによりダウンロードが成功したかどうかを調べられます。もっと簡単に成功したかどうかを調べるには `Response` オブジェクトの `raise_for_status` メソッドを呼び出します。

このメソッドはファイルのダウンロードが失敗すれば例外を発生させ，成功すればなにもしません。

例えば、次の例では example.com という存在しないURLに対してリクエストを送っています。以下を実行すると HTTPError が表示されますが、これは正常な挙動です。

In [ ]:
res = requests.get('https://www.example.com/download-helloworld')
res.raise_for_status()

`raise_for_status()` メソッドは，ダウンロードが失敗したら必ずプログラムを停止するのにとてもよい方法です。予期せぬエラーが起きたらすぐにプログラムを停める方がよい場合が多いからです。

ダウンロード失敗がプログラムを停止させるほどのものでないなら `raise_for_status()` の行を [try/except](https://docs.python.org/ja/3/tutorial/errors.html) 文で囲むことにより，異常終了せずにエラーを処理できます。Pythonの例外についての詳しい説明は省きますが，興味のある方はリンクから公式ドキュメントを読んだり，調べたりしてみてください。

In [ ]:
res = requests.get('https://www.example.com/download-helloworld')
try:
    res.raise_for_status()
except Exception as exc:
    print(f'問題あり: {exc}')

`requests.get()` を呼び出したら，必ず `raise_for_status()` を呼び出すようにしましょう。プログラムの実行を継続する前に，実際にダウンロードされたことを確認する方がよいからです。

# HTML

実際にWebページを解析する前に，HTMLの基本について学びましょう。またWebブラウザの強力な開発者向けツールの操作方法を覚えて，簡単にWebからの情報をスクレイピングできるようにしましょう。

## HTMLについて学習するには

**HTML** (HyperText Markup Language) は，Webページを記述するための記法です。このStationではHTMLの基本を知っていることを前提にしています。もしHTMLに明るくない方は，次のサイトをおすすめします。

- https://developer.mozilla.org/ja/docs/Learn/HTML (日本語)
- https://htmldog.com/guides/html/beginner/ (英語)
- https://www.codecademy.com/learn/learn-html/ (英語)

## HTMLについてざっとおさらい

HTMLについて調べてからしばらく間がある方もいることでしょう。ここでは簡単に基本をおさらいします。HTMLファイルはプレーンテキストであり，一般に `.html` や `.htm` といった拡張子を持ちます。

テキストは `<>` で記述された **タグ (Tag)** で囲まれています。タグはWebページの構成や成形方法をブラウザに伝えます。開始タグ `<>` と終了タグ `</>` でテキストを囲むと **要素** になります。 **内部テキスト** とは，開始タグと終了タグで囲まれた内容のことです。たとえば，次のHTMLはブラウザに "Hello, world!" と表示するものですが `<strong>` の指定により "Hello" の部分を強調表示します。

```html
 <strong>Hello</strong>, world!
 ```

 このHTMLはブラウザ上では以下のように表示されます。

 <img src="https://drive.google.com/uc?id=1NjmBhGf1j67TWXl244HeeZxIFh_ks-R7" width="480px">

開始タグ `<strong>` は囲んだテキストを強調表示することを意味します。そして終了タグ `</strong>` は強調表示するテキストの終端を表します。

HTMLにｋはこれ以外にもさまざまなタグがあります。タグには `<` と `>` の中に **属性** を指定できるものもありまし。たとえば `<a>` タグはリンクとなるテキストを囲むものですが，リンク先のURLは `href` 属性に記述します。

```html
 Al's free <a href="https://inventwithpython.com">Python books</a>.
```

このHTMLをブラウザで表示すると以下のようになります。

<img src="https://drive.google.com/uc?id=1d0TxG80pj_5NL7ttxpp2e1GFyoxIktlE" width="480px">

要素には `id` 属性をつけてページ上の要素を識別するのに使えます。Webスクレイピングのプログラムを書くときには `id` 属性を用いて要素を探すプログラムを組むことがよくあります。そこで，ブラウザの開発者ツールを使って属性の `id` を探すと便利です。

## WebページのソースHTMLを見る

プログラムの処理対象となるWebページのソースHTMLを調べるには，Webブラウザのぺージ上で右クリック (macOSでは `Ctrl` キーを押しながらクリック) し [ページのソースを表示] のメニュー項目を選びます。するとブラウザが実際に取得したテキストが表示されます。このHTMLに基づいて，ブラウザはWebページを整形して表示します。

<img src="https://drive.google.com/uc?id=1Qrw5zpGTJ-qfgdZ5IX4voBrCvzBA6XPf" width="480px">

<img src="https://drive.google.com/uc?id=1Z16YsRKFg6yMaXHUpab6arkx0oasYxGL" width="480px">

お気に入りのWebサイトのソースHTMLを見てみましょう。ソースの内容を完全に理解する必要はありません。簡単なWebスクレイピングプログラムを書くのに，HTMLに熟達する必要はないのです。Webサイトを作るわけではないのですから，既存のサイトからデータを取り出すために必要な知識があれば十分です。

## ブラウザの開発者ツールを開く

Webページのソースを見るだけでなく，開発者ツールを使ってHTMLを解析してみましょう。ChromeやEdgeには開発者ツールが内蔵されていて， `F12` キーを押すと表示されます。もう一度 `F12` キーを押すと開発者ツールは消えます。Chromeの場合では **メニュー** → **その他のツール** → **デベロッパーツール** や `Ctrl - Shift - I` でも開発者ツールを表示できます。macOSでは `⌘ - Option - I` でChromeの開発者ツールが開きます。FirefoxやSafariにも同様の機能がありますが，ここでは詳しく書きません。利用している人は自分で調べてみてください。

ブラウザの開発者ツールを使うと，Webページ上のありとあらゆる場所を右クリックして **要素の調査** や **検証** を行い，対応するHTML要素を表示できます。Webスクレイピングをするプログラムを作るとき，HTML解析に着手するのに便利です。

### HTMLを解析するのに正規表現を使わない

正規表現を学んだみなさんなら，文字列に格納されたHTMLから特定の部分を見つけるのに正規表現はうってつけに見えます。しかし現実では正規表現はこの用途ではあまり使われません。HTMLの書き方にはさまざまな方法があり，いずれも正しいHTMLとみなされます。そのため可能なバリエーションすべてに対応しようとするのは厄介で間違いが起こりやすいです。Beautiful SoupのようなHTMLを解析するモジュールを使えば，バグになりにくくなります。

## 開発者ツールを使ってHTML要素を検索する

次に必要なのは，Webページ上で関心のある情報に対応するのはHTMLのどの部分なのか見つけることです。

ここでブラウザの開発者ツールが役に立ちます。例として https://tenki.jp/ から天気予報のデータを取り出すプログラムを書いてみます。コードを書く前に少し調べましょう。ブラウザでこのサイトを開いて郵便番号100-0001を探すと，その地域の天気予報を表示します。

ここでは，郵便番号に対応する天気情報を取り出すことにしましょう。 [千代田区の天気](https://tenki.jp/forecast/3/16/4410/13101/) のページ上の天気アイコンの箇所で右クリックして，メニューから「要素を調査」や「検証」を選択してください。開発者ツールが開いて，対応するHTMLを表示します。

<img src="https://drive.google.com/uc?id=13RnI2W0SWe9PHEhVbM2BhKEKh5jWt6d6" width="720px">

なお，Webサイト https://tenki.jp/ のデザインが変わったら，以下の手順をやり直して要素を確認する必要があります。

開発者ツールを見ると，Webページ上で **今日の天気** に対応するHTMLは次の箇所です。

```html
<p class="weather-telop">曇のち晴</p>
```

これがまさに探していた情報です。天気予報の情報は `<div>` 要素の中に含まれており `weather-icon` というCSSクラスを持つようです。開発者ツールをこの要素の上で右クリックして `Copy` → `CSS Selector` または `Copy selector` を選ぶと，次のような文字列をクリップボードにコピーできます。

```css
#main-column > section > div.forecast-days-wrap.clearfix > section.today-weather > div.weather-wrap.clearfix > div.weather-icon > p
```

この文字列はCSSセレクタといって，後述のBeautiful Soupの `select()` やSeleniumの `find_element(By.CSS_SELECTOR, ...)` に渡してHTML要素を取得できます。それではBeautifulSoupを用いて文字列を取得してみましょう。

# bs4モジュールを使ってHTMLを解析する

Beautiful Soup はHTMLページから情報を抽出するモジュールで，この目的では正規表現よりも優れています。Beautiful Soup のモジュール名は `bs4` (Beautiful Soup Version 4) です。

このStationでは Beautiful Soup によってHTMLファイルを **パース (Parse)** することを学びます。パースとは，部分を識別できるように構文解析することです。構文解析器のことを **パーサー (Parser)** と呼びます。

単純に見えるHTMLでも，実際にはさまざまなタグが使われています。複雑なWebサイトであれば，すぐにもっと込み入ったものになります。このような場合に Beautiful Soup を使うとHTMLを簡単に扱えて非常に便利です。

In [ ]:
import requests, bs4
res = requests.get('https://tenki.jp')
res.raise_for_status()
tenkijp = bs4.BeautifulSoup(res.content, 'html.parser')
type(tenkijp)

## HTMLからBeautifulSoupオブジェクトを生成する

このコードでは `requests.get()` を用いて tenki.jp のウェブサイトからページをダウンロードし，レスポンスの `content` 属性を `bs4.BeautifulSoup()` に渡しています。 `BeautifulSoup` オブジェクトは，変数 `tenkijp` に格納されています。

`BeautifulSoup()` の第2引数には，解析するパーサーを指定します。 `html.parser` は Python に付属しているものです。この他にサードパーティーの高速な `lxml` パーサーを使うことも可能です。

`BeautifulSoup` オブジェクトを取得できたら，そのメソッドを用いてHTMLの特定の部分を見つけられます。

## `select()` メソッドを用いて要素を見つける

`BeautifulSoup` オブジェクトの `select()` メソッドに，検索したい要素の **CSSセレクタ (CSS Selector)** を渡して呼び出すと，Webページの要素を取得できます。セレクタとは，HTMLページから検索対象を指定するためのパターンのことです。テキストから検索するための正規表現のようなものです。

CSSセレクタの文法をすべて説明するのは困難なので，ここではセレクタを簡単に説明するだけにとどめます。以下によく使われるCSSセレクタのパターンを示します。

|  select() に渡すセレクタ  |  マッチする対象  |
| ---- | ---- |
|  `soup.select('div')`  |  すべての `<div>` 要素  |
|  `soup.select('#author')`  |  `id` 属性が `author` である要素  |
|  `soup.select('.notice')`  | CSSクラス属性が `notice` である全要素  |
|  `soup.select('div span')`  | `<div>` 要素の中のすべての `<span>` 要素  |
|  `soup.select('div > span')` | `<div>` 要素の直下のすべての `<span>` 要素 (間に他の要素がない) |
|  `soup.select('input[name]')` | `name` 属性 (値は任意) を持つすべての `<input>` 要素 |
|  `soup.select('input[type=button]')` | `type` 属性の値が `button` であるすべての `<input>` 要素 |

さまざまなセレクタのパターンを組み合わせることで，複雑な検索が可能です。例えば `soup.select('p #author')` は `<p>` 属性の中にあって `id` 属性が `author` である要素にマッチします。セレクタを自分で書くのではなく，先に書いたとおりにブラウザ上で右クリックして「要素の調査」をし，開発ツール上のCSSセレクタをクリップボードにコピーして，ソースコードに貼り付けることもできます。

`select()` メソッドは `Tag` オブジェクトのリストを返します。 `Tag` オブジェクトは Beautiful Soup におけるHTML要素の表現です。リストには `BeautifulSoup` オブジェクトのHTMLの中でマッチしたすべての要素が含まれています。 `Tag` オブジェクトは `str()` 関数に渡して，それが表現する要素を文字列として取得できます。 `Tag` オブジェクトには `attrs` 属性もあって，タグのすべての属性を辞書として保持しています。

In [10]:
import bs4
res = requests.get('https://lp.techbowl.co.jp/')
res.raise_for_status()
example_soup = bs4.BeautifulSoup(res.content)

このコードではTechTrainの紹介LPから `id="background"` である要素を取り出しています。`select('#background')` を呼び出すと `id="background"` であるすべての要素のリストが返ってきます。

In [11]:
elems = example_soup.select('#background')

In [ ]:
type(elems[0])

`elems` は `Tag` オブジェクトのリストです。この `Tag` オブジェクトのリストを変数 `elems` に格納し `len(elems)` を見てみましょう。

In [ ]:
len(elems)

ここで `1` が返ってきますね。つまりマッチしたのは1つであったことがわかります。

In [ ]:
elems[0].getText()

要素の `getText()` メソッドを呼び出すと，要素の内部テキストを取得できます。内部テキストは開始タグと終了タグの間の内容なので，この場合は `優秀なエンジニアって…` からはじまる文章が取れることがわかります。

In [ ]:
str(elems[0])

要素を `str()` にわたすと，開始タグと終了タグを含む文字列を返します。

In [ ]:
elems[0].attrs

最後に `attrs` は要素の属性を辞書として保存しています。この場合は `id` とその値 `background` を持つ辞書になります。また見た目を表すためのクラスがいくつも指定されていることがわかります。

`BeautifulSoup` オブジェクトからすべての `<p>` 要素を取り出すことも可能です。

In [17]:
p_elems = example_soup.select('p')

この例では `select()` は88個の要素からなるリストを返します。これを `p_elems` に格納し `p_elems[0]` から `p_elems[2` を `str()` に渡すと，各要素を文字列として取得できます。また `getText()` を呼び出すと，内部テキストを取得できます。

In [ ]:
len(p_elems)

In [ ]:
str(p_elems[0])

In [ ]:
p_elems[0].getText()

In [ ]:
str(p_elems[1])

In [ ]:
p_elems[1].getText()

In [ ]:
str(p_elems[2])

In [ ]:
p_elems[2].getText()

## 要素の属性からデータを取得する

`Tag` オブジェクトの `get()` メソッドを用いると，要素の属性値を取得するのが簡単です。このメソッドに属性値を渡すと，その属性値を返します。

In [25]:
import bs4
res = requests.get('https://lp.techbowl.co.jp/')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.content)

In [26]:
span_elem = soup.select('span')[0]

In [ ]:
str(span_elem)

In [ ]:
span_elem.get('class')

In [ ]:
span_elem.get('non_ecistent_attribute') == None

In [ ]:
span_elem.attrs

# 確認テスト

[TechTrainのMission一覧ページ](https://techtrain.dev/missions) を使って実際にスクレイピングをしてみましょう。

1. 企業Missionは全部で何件ありますか？
2. 企業Missionのうち名前に "iOS" を含むものはいくつありますか？
3. Missionには `HTML` や `JavaScript` といったタグがついています。タグは全部で何種類ありますか？重複を削除した上で答えてください。

In [ ]:
import bs4
res = requests.get('https://techtrain.dev/missions')
res.raise_for_status()
results = bs4.BeautifulSoup(res.content, 'html.parser')
type(results)

In [ ]:
print(results)

In [33]:
m = results.select(".mt-1")

In [ ]:
print(m)

In [ ]:
len(m)

In [36]:
txt = results.getText()

In [37]:
text = str(txt)

In [ ]:
print(text)

In [ ]:
text.count("iOS")

In [ ]:
tag_types = set()
for tag in results.find_all():
    tag_types.add(tag.name)

print(len(tag_types))